In [13]:
#!/usr/bin/env python3
import sys
from pathlib import Path
from spicelib import AscEditor                    # for working with .asc files :contentReference[oaicite:0]{index=0}
from spicelib.simulators.ltspice_simulator import LTspice  # for netlist generation :contentReference[oaicite:1]{index=1}

def display_netlist(asc_file: str):
    asc_path = Path(asc_file)
    net_path = asc_path.with_suffix(".net")

    # If LTspice isn’t on your PATH, point spicelib at the executable:
    # LTspice.spice_exe = r"C:\Program Files\LTC\LTspiceXVII\XVIIx64.exe"

    # Generate the .net (or .cir) file from the schematic
    # (instead of save_netlist which only writes .asc) :contentReference[oaicite:2]{index=2}
    LTspice.create_netlist(str(asc_path), str(net_path))

    # Read and print the generated netlist
    print(net_path.read_text())

if __name__ == "__main__":
    asc_path = r"C:\Users\inxtc\Documents\01_仕事関連\09_仕事代理\002_移動平均スクリプト\series_rlc.asc"
    display_netlist(asc_path)


* C:\Users\inxtc\Documents\01_仕事関連\09_仕事代理\002_移動平均スクリプト\series_rlc.asc
R1 N001 node1 R
L1 node1 N002 L
C1 N002 node2 C
V1 N001 node2 1
.tran 1m
.backanno
.end



In [3]:
import sys
import clipboard
clipboard.copy(sys.executable)

In [ ]:
from PyLTSpice import SimCommander
from pathlib import Path
asc_file = r"series_rlc.asc"
asc_path = Path(asc_file)
# ネットリストを表示
net_path = asc_path.with_suffix(".net")
print(net_path.read_text())


* c:\Users\inxtc\Documents\01_仕事関連\09_仕事代理\002_移動平均スクリプト\series_rlc.asc
R1 N001 node1 100
L1 node1 N002 100u
C1 N002 0 10u
V1 N001 0 PULSE(0 1 0 1u 1u 100u 200u 10)
.tran 1m
.backanno
.end



In [ ]:
import os
import pandas as pd
from functools import reduce
from PyLTSpice import SimCommander, RawRead
import plotly.graph_objects as go

# 1) 元の.ascファイルを読み込み（同時に.netも生成）
LTC = SimCommander("series_rlc.asc")
print("".join(LTC.netlist))


# 2) スイープするR1の値リスト
values = ["50k", "20k", "5k", "1k"]

# 実行したシミュレーション情報を保持
run_files = []

# 3) 各値でシミュレーション実行
for i, val in enumerate(values):
    LTC.set_component_value("R1", val)
    run_net_file = f"series_rlc_{i}.net"
    LTC.run(run_filename=run_net_file)
    run_files.append((val, run_net_file))

# 4) 全シミュレーションの終了を待機
LTC.wait_completion()

# 5) 各.rawファイルを読み込んでDataFrame化
dfs = []
for val, net_file in run_files:
    raw_path = os.path.splitext(net_file)[0] + ".raw"
    raw = RawRead(raw_path)

    # トレース名を自動検出
    traces = raw.get_trace_names()
    trace_name = 'V(node1)' if 'V(node1)' in traces else next(t for t in traces if t.startswith('V('))

    # 時間と電圧データを取得
    time = raw.get_axis()
    voltage = raw.get_trace(trace_name).get_wave()

    # DataFrame に格納
    df = pd.DataFrame({
        "time": time,
        f"V_node1_R1={val}": voltage
    })
    dfs.append(df)

# 6) time 列でマージして，一つの DataFrame にまとめる
combined_df = reduce(lambda left, right: pd.merge(left, right, on="time"), dfs)

# 確認
print(combined_df.head())

# 7) （オプション）Plotly で描画
fig = go.Figure()
for col in combined_df.columns:
    if col != "time":
        fig.add_trace(go.Scatter(
            x=combined_df["time"],
            y=combined_df[col],
            mode="lines",
            name=col
        ))
fig.update_layout(
    title="Node1 Voltage Sweep",
    xaxis_title="Time [s]",
    yaxis_title="Voltage [V]",
    # template="simple_white"
)
fig.show()


       time  V_node1_R1=50k  V_node1_R1=20k  V_node1_R1=5k  V_node1_R1=1k
0  0.000000        0.000000        0.000000       0.000000       0.000000
1  0.000001        0.002001        0.005002       0.020011       0.100034
2  0.000101        0.000228        0.000466       0.002007       0.009981
3  0.000102       -0.001727       -0.004191      -0.017992      -0.089954
4  0.000200        0.000202        0.000505       0.002014       0.009950


In [ ]:
import os
import pandas as pd
from functools import reduce
from PyLTSpice import SimCommander, RawRead
import plotly.graph_objects as go
import numpy as np

def calc_thermal_resistance_cylinder(Ri, Ro, L, k):
    """
    円筒の熱抵抗を計算する関数
    :param Ri: 内半径 [m]
    :param Ro: 外半径 [m]
    :param L: 長さ [m]
    :param k: 熱伝導率 [W/(m*K)]
    :return: 熱抵抗 [K/W]
    """
    return np.log(Ro/Ri) / (2 * np.pi * k * L)

def calc_thermal_convection_equivalent_resistance(Ro, h = 10, L=1):
    """
    熱対流の等価熱抵抗を計算する関数
    :param Ro: 外半径 [m]
    :param h: 熱伝達率 [W/(m^2*K)]
    :param L: 長さ [m] (省略可能、デフォルトは1m)
    :return: 熱抵抗 [K/W]
    R = 1 / (h * S) = 1 / (h * 2 * π * Ro * L)
    """
    # 空気の自然対流における熱伝達係数
    # h = 10  # W/(m^2*K) の例（実際の値は条件による）
    return 1 / (h * 2 * np.pi * Ro * L)

def calc_forced_convection_equivalent_resistance(Ro, Ri=1.0, h=100, L=1, model="Equivalent Value"):
    """
    強制対流の等価熱抵抗を計算する関数
    :param Ro: 外半径 [m]
    :param h: 熱伝達率 [W/(m^2*K)]
    :param L: 長さ [m] (省略可能、デフォルトは1m)
    :return: 熱抵抗 [K/W]
    R = 1 / (h * S) = 1 / (h * 2 * π * Ro * L)
    """
    if model == "Equivalent Value":
        # 強制対流の等価熱抵抗
        h = 100  # W/(m^2*K) の例（実際の値は条件による）
        R = 1 / (h * 2 * np.pi * Ro * L)
    else:
        # 熱輸送モデルによる等価熱抵抗
        Q = np.pi * (Ro**2 - Ri**2) * L  # 円筒の断面積
        R = 1 / (1150 * Q)
    return R

def define_thermal_resistances(idx=1, thermal_type="Conduction", Ri=0.01, Ro=0.012, L=0.1, k=0.1, h=10, model="Equivalent Value"):
    """
    熱抵抗の定義を行う関数
    :return: 熱抵抗のリスト
    """
    if thermal_type == "Conduction":
        # 熱伝導の熱抵抗
        val_dict = {
            "Name": f"R{idx}",
            "Type": thermal_type, 
            "Ri": Ri, # 内半径 [m]
            "Ro": Ro, # 外半径 [m]
            "L": L, # 円筒の長さ [m]
            "k": k, # 熱伝導率 [W/(m*K)]
            "Rt": calc_thermal_resistance_cylinder(Ri, Ro, L, k)
        }
    elif thermal_type == "N-Convection":
        # 自然対流の等価熱抵抗
        val_dict = {
            "Name": f"R{idx}",
            "Type": thermal_type, 
            "Ri": Ri, # 内半径 [m] 
            "Ro": Ro, # 外半径 [m]
            "L": L, # 円筒の長さ [m]
            "h": 10,  # 熱伝達率の例
            "Rt": calc_thermal_convection_equivalent_resistance(Ro, h=10, L=L)
        }
    elif thermal_type == "F-Convection":
        # 強制対流の等価熱抵抗
        val_dict = {
            "Name": f"R{idx}",
            "Type": thermal_type, 
            "Ri": Ri, # 内半径 [m] 
            "Ro": Ro, # 外半径 [m] 
            "L": L, # 平均風速 [m/s]
            "h": 100,  # 熱伝達率の例
            "model": model,  # モデルの指定
            "Rt": calc_forced_convection_equivalent_resistance(Ro, Ri=Ri, h=h, L=L, model=model)
        }
    return val_dict

# 1) 元の.ascファイルを読み込み（同時に.netも生成）
LTC = SimCommander("Equivalent_ThermalCircuit.asc")
print("".join(LTC.netlist))

# 2) パラメータの設定値リスト
# === １．与件（実際の値を入れてください） ===
k_Cu = 400 # 銅の熱伝導率 [W/(m*K)] 385 ~ 400
k_Coil = 1.5  # 銅コイル（被覆あり）の平均熱伝導率 [W/(m*K)] 1 ~ 2
k_PC = 0.19 # ポリカーボネートの熱伝導率 [W/(m*K)] 0.19 ~ 0.20
k_Air = 0.025  # 空気の熱伝導率 [W/(m*K)] 0.025 ~ 0.026
h_naturalair = 10  # 自然対流の熱伝達率 [W/(m^2*K)] 10 ~ 20
h_forcedair = 100  # 強制対流の熱伝達率 [W/(m^2*K)] 100 ~ 200
v_forcedair = 1.0  # 強制対流の平均風速 [m/s] 1 ~ 2
v_gapair = 0.001  # ギャップからの換気風速
# === ２．コイルの寸法 ===
Rrx1 = 0.01  # コイル内面の半径 [m]
Rrx2 = 0.012  # コイル内コイル層の半径 [m]
Rrx3 = 0.014  # コイル外コイル層の半径 [m]
Rtx1 = 0.016  # コイル外面の半径 [m]
Rtx2 = 0.018  # コイル外面の外側の半径 [m]
Rtx3 = 0.02  # コイル外面の外側の外側の半径 [m]
Rb11 = 0.022  # コイル外面の外側の外側の外側の半径 [m]
Rb12 = 0.024  # コイル外面の外側の外側の外側の半径 [m]
Rb13 = 0.026  # コイル外面の外側の外側の外側の外側の半径 [m]
Rb21 = 0.028  # コイル外面の外側の外側の外側の外側の外側の半径 [m]
Rb22 = 0.03  # コイル外面の外側の外側の外側の外側の外側の外側の半径 [m]
Rb23 = 0.032  # コイル外面の外側の外側の外側の外側の外側の外側の半径 [m]
Rb31 = 0.034  # コイル外面の外側の外側の外側の外側の外側の外側の外側の半径 [m]
Rb32 = 0.036  # コイル外面の外側の外側の外側の外側の外側の外側の外側の半径 [m]
Rb33 = 0.038  # コイル外面の外側の外側
L_coil = 0.20  # コイルの長さ [m]（全てのコイルで同じ長さとする）
# === ３．発熱量 ===
I_tx = 1.0  # Txコイルの発熱量 [W]
I_bx1 = 1.0  # Biasコイル1の発熱量 [W]
I_bx2 = 1.0  # Biasコイル2の発熱量 [W]
I_bx3 = 1.0  # Biasコイル3の発熱量 [W]

# 熱抵抗リスト
thermal_resistances = [
    define_thermal_resistances(idx=1,   thermal_type="N-Convection",    Ri=Rrx1, Ro=Rrx1, L=L_coil, k=k_Air, h=h_naturalair),  # R1 
    define_thermal_resistances(idx=2,   thermal_type="Conduction",      Ri=Rrx1, Ro=Rrx2, L=L_coil, k=k_PC),  # コイル内面－コイル内コイル層    
    define_thermal_resistances(idx=3,   thermal_type="Conduction",      Ri=Rrx2, Ro=Rrx3, L=L_coil, k=k_Coil),  # コイル内面－コイル内コイル層    M
    define_thermal_resistances(idx=4,   thermal_type="N-Convection",    Ri=Rrx3, Ro=Rrx3, L=L_coil, k=k_Air, h=h_naturalair),  # コイル内面－コイル内コイル層    M
    define_thermal_resistances(idx=5,   thermal_type="N-Convection",    Ri=Rtx1, Ro=Rtx1, L=L_coil, k=k_Air, h=h_naturalair),  # コイル内面－コイル内コイル層    M
    define_thermal_resistances(idx=6,   thermal_type="Conduction",      Ri=Rtx1, Ro=Rtx2, L=L_coil, k=k_PC),  # コイル内面－コイル内コイル層    M
    define_thermal_resistances(idx=7,   thermal_type="Conduction",      Ri=Rtx2, Ro=(Rtx2+Rtx3)/2, L=L_coil, k=k_Coil),  # コイル内面－コイル内コイル層    M
    define_thermal_resistances(idx=8,   thermal_type="Conduction",      Ri=(Rtx2+Rtx3)/2, Ro=Rtx3, L=L_coil, k=k_Coil),  # コイル内面－コイル内コイル層    M
    define_thermal_resistances(idx=9,   thermal_type="N-Convection",    Ri=Rtx3, Ro=Rtx3, L=L_coil, k=k_Air, h=h_naturalair),  # コイル内面－コイル内コイル層    M
    define_thermal_resistances(idx=10,  thermal_type="N-Convection",    Ri=Rb11, Ro=Rb11, L=L_coil, k=k_Air, h=h_naturalair),  # コイル内面－コイル内コイル層   
    define_thermal_resistances(idx=11,  thermal_type="Conduction",      Ri=Rb11, Ro=Rb12, L=L_coil, k=k_Cu),  # コイル内面－コイル内コイル層   
    define_thermal_resistances(idx=12,  thermal_type="Conduction",      Ri=Rb12, Ro=(Rb12+Rb13)/2, L=L_coil, k=k_Coil),  # コイル内面－コイル内コイル層    M
    define_thermal_resistances(idx=13,  thermal_type="Conduction",      Ri=(Rb12+Rb13)/2, Ro=Rb13, L=L_coil, k=k_Coil),  # コイル内面－コイル内コイル層    M
    define_thermal_resistances(idx=14,  thermal_type="N-Convection",    Ri=Rb13, Ro=Rb13, L=L_coil, k=k_Air, h=h_naturalair),  # コイル内面－コイル内コイル層    M
    define_thermal_resistances(idx=15,  thermal_type="N-Convection",    Ri=Rb21, Ro=Rb21, L=L_coil, k=k_Air, h=h_naturalair),  # コイル内面－コイル内コイル層   
    define_thermal_resistances(idx=16,  thermal_type="Conduction",      Ri=Rb21, Ro=(Rb21+Rb22)/2, L=L_coil, k=k_Coil),  # コイル内面－コイル内コイル層   
    define_thermal_resistances(idx=17,  thermal_type="Conduction",      Ri=(Rb21+Rb22)/2, Ro=Rb22, L=L_coil, k=k_Coil),  # コイル内面－コイル内コイル層    M
    define_thermal_resistances(idx=18,  thermal_type="N-Convection",    Ri=Rb22, Ro=Rb22, L=L_coil, k=k_Air, h=h_naturalair),  # コイル内面－コイル内コイル層    M
    define_thermal_resistances(idx=19,  thermal_type="N-Convection",    Ri=Rb31, Ro=Rb31, L=L_coil, k=k_Air, h=h_naturalair),  # コイル内面－コイル内コイル層   
    define_thermal_resistances(idx=20,  thermal_type="Conduction",      Ri=Rb31, Ro=(Rb31+Rb32)/2, L=L_coil, k=k_Coil),  # コイル内面－コイル内コイル層   
    define_thermal_resistances(idx=21,  thermal_type="Conduction",      Ri=(Rb31+Rb32)/2, Ro=Rb32, L=L_coil, k=k_Coil),  # コイル内面－コイル内コイル層    M
    define_thermal_resistances(idx=22,  thermal_type="N-Convection",    Ri=Rb32, Ro=Rb32, L=L_coil, k=k_Air, h=h_naturalair),  # コイル内面－コイル内コイル層    M
    define_thermal_resistances(idx=23,  thermal_type="F-Convection",    Ri=Rb22, Ro=Rb31, L=v_forcedair, k=k_Air, h=h_naturalair, model="vent model"),  # コイル内面－コイル内コイル層   
    define_thermal_resistances(idx=24,  thermal_type="F-Convection",    Ri=Rb13, Ro=Rb21, L=v_forcedair, k=k_Air, h=h_naturalair, model="vent model"),  # コイル内面－コイル内コイル層   
    define_thermal_resistances(idx=25,  thermal_type="F-Convection",    Ri=0, Ro=Rrx1, L=v_gapair, k=k_Air, h=h_naturalair, model="vent model"),  # コイル内面－コイル内コイル層   
]

values = ["50k", "20k", "5k", "1k"]

# # 実行したシミュレーション情報を保持
# run_files = []

# # 3) 各値でシミュレーション実行
# for i, val in enumerate(values):
#     LTC.set_component_value("R1", val)
#     run_net_file = f"series_rlc_{i}.net"
#     LTC.run(run_filename=run_net_file)
#     run_files.append((val, run_net_file))

# # 4) 全シミュレーションの終了を待機
# LTC.wait_completion()

# # 5) 各.rawファイルを読み込んでDataFrame化
# dfs = []
# for val, net_file in run_files:
#     raw_path = os.path.splitext(net_file)[0] + ".raw"
#     raw = RawRead(raw_path)

#     # トレース名を自動検出
#     traces = raw.get_trace_names()
#     trace_name = 'V(node1)' if 'V(node1)' in traces else next(t for t in traces if t.startswith('V('))

#     # 時間と電圧データを取得
#     time = raw.get_axis()
#     voltage = raw.get_trace(trace_name).get_wave()

#     # DataFrame に格納
#     df = pd.DataFrame({
#         "time": time,
#         f"V_node1_R1={val}": voltage
#     })
#     dfs.append(df)

# # 6) time 列でマージして，一つの DataFrame にまとめる
# combined_df = reduce(lambda left, right: pd.merge(left, right, on="time"), dfs)

# # 確認
# print(combined_df.head())

# # 7) （オプション）Plotly で描画
# fig = go.Figure()
# for col in combined_df.columns:
#     if col != "time":
#         fig.add_trace(go.Scatter(
#             x=combined_df["time"],
#             y=combined_df[col],
#             mode="lines",
#             name=col
#         ))
# fig.update_layout(
#     title="Node1 Voltage Sweep",
#     xaxis_title="Time [s]",
#     yaxis_title="Voltage [V]",
#     # template="simple_white"
# )
# fig.show()


* c:\Users\inxtc\Documents\01_仕事関連\09_仕事代理\002_移動平均スクリプト\Equivalent_ThermalCircuit.asc
R1 N004 N003 1
R2 N005 N004 1
R3 N006 N005 1
R4 N007 N006 1
R5 N008 N007 1
R6 N009 N008 1
R7 N010 N009 1
R8 N011 N010 1
R9 N012 N011 1
R10 N013 N012 1
R11 N014 N013 1
R12 N015 N014 1
R13 N016 N015 1
R14 N017 N016 1
R15 N001 N017 1
R16 N018 N001 1
R17 N019 N018 1
R18 N020 N019 1
R19 N002 N020 1
R20 N021 N002 1
R21 N022 N021 1
R22 N023 N022 1
R23 0 N023 1
R24 0 N002 1
R25 0 N001 1
R26 0 N003 1
I1 0 N011 1
I2 0 N016 1
I3 0 N018 1
I4 0 N022 1
.tran 1m
.backanno
.end



In [27]:
import numpy as np
import plotly.graph_objects as go

delta_T = np.linspace(0, 100, 100)  # 温度差の範囲
# 熱伝達率 h = 2.51 * C * (delta_T/L) ** 0.25
C = 0.52
L = 0.01  # 長さ [m]
h = 2.51 * C * (delta_T/L) ** 0.25

# Plotly で描画
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=delta_T,
    y=h,
    mode='lines',
    name='Heat Transfer Coefficient',
    line=dict(color='blue')
))